In [23]:
import pandas as pd

In [24]:
inname = 'LabelsWithExtraInfo_adni3_train.csv'
outname = 'Adni3_trainLabels_extensive2.txt'

In [25]:
CreateLabelsFrame = pd.read_csv(inname)
CreateLabelsFrame = CreateLabelsFrame.drop(['dicomPath','info','xmlName'],axis=1)
CreateLabelsFrame.head()

,ID,label,extra,description,weighting
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0,Sagittal_3D_FLAIR,T2
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0,Sagittal_3D_FLAIR,T2
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0,Axial_T2_Star,T2
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0,HighResHippo_Scan__Oblique_-_perpendicular_to_...,PD
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0,Sagittal_3D_Accelerated_MPRAGE,T1


In [26]:
CreateLabelsFrame['origDesc'] = CreateLabelsFrame['description']

In [27]:
CreateLabelsFrame = CreateLabelsFrame.dropna()

In [28]:
uniqueDesc = CreateLabelsFrame[CreateLabelsFrame['weighting']=='PD']['description'].unique()
print(uniqueDesc)

['HighResHippo_Scan__Oblique_-_perpendicular_to_hippocampal_tail'
 'Axial_2D_PASL' 'HighResHippocampus' 'HighResHippo' 'Cerebral_Blood_Flow'
 'Axial_2D_PASL_straight_no_ASL' 'Axial_DTI_straight' 'Perfusion_Weighted'
 'tgse_pcasl_PLD2000_phase_A-P' 'HighResHippocampus_Phase_R-L'
 'tgse_pcasl_PLD2000' 'Axial_DTI' 'ORIG_HighResHippo'
 'Axial_2D_PASL_0_angle_L' 'Accelerated_Sagittal_MPRAGE_Phase_A-P' 'T2_AX'
 'relCBF' 'MoCoSeries' 'HighResHippocampus_REPEAT' 'sPWI'
 'Axial_3D_pCASL_Eyes_Open'
 'Axial_3D_PASL__Eyes_Open_____straight_no_angle' 'Axial_MB_DTI'
 'Axial_MB_DTI_phase_noFatSatA' 'Accelerated_Sagittal_MPRAGE_MPR_Tra'
 'Accelerated_Sagittal_MPRAGE_MPR_Cor' 'Accelerated_Sagittal_MPRAGE'
 'sPerfusionWeighted' 'COR_HighResHippo' 'Accelerated_Sagittal_MPRAGE_ND'
 'HighResHippocampus_repeat' 'SOURCE_-_Axial_2D_PASL'
 'HighResHippo_Scan_REPEAT' 'WIP_SOURCE_-_Axial_2D_PASL'
 'HighResHippocampus_Phase_R-L_rpt' 'Obl_HighResHippo'
 'Axial_3D_PASL__Eyes_Open__REPEAT' 'Axial_3D_pCASL'
 'HighRes

In [29]:
def renameDesc(desc):
    desc = desc.lower()
    if 'localizer' in desc:
        return 'localizer'
    if 'phase' in desc:
        return 'phase'
    if 'field' in desc:
        return 'fieldmap'
    if 'rage' in desc or 'fspgr' in desc:
        return 'T1w'
    if 'cal' in desc:
        return 'calibration'
    if 'scout' in desc:
        return 'scout'
    if 'flair' in desc:
        return 'T2-FLAIR'
    if 'star' in desc:
        return 'T2star'
    if 'swi' in desc:
        return 'swi'
    if 'hippo' in desc:
        return 'hippo'
    if 'perf' in desc or 'pwi' in desc:
        return 'pwi'
    if 'asl' in desc:
        return 'asl'
    if 'dti' in desc:
        return 'dti'
    if 'cerebral_blood_flow' in desc or 'cbf' in desc:
        return 'cbf'
    if 't2' in desc:
        return 'T2w'
    if 'moco' in desc:
        return 'moco'
    if 'format' in desc:
        return 'mpr'
    if 'coronal' in desc:
        return 'T1w'


In [30]:
CreateLabelsFrame['description'] = CreateLabelsFrame['origDesc'].apply(renameDesc)

In [31]:
CreateLabelsFrame[CreateLabelsFrame['description'].isna()]

,ID,label,extra,description,weighting,origDesc


In [32]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"FS":4,"PD":5,
             "SWI":6, "GRE":7, "T2*":8, "DWI":9, "ADC":10, "BOLD":11,
             "FlowSensitve":12,"PWI":13,"ASL":14,"Hippo":15,
             "DTI":16,"Other":17}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP','moco','mpr'] #moco is a motion corrected series
    if description in otherCategories:
        return 17
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 5
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='FS':
        return 4
    if description =='pd':
        return 5
    if description =='swi':
        return 6
    if description.lower() =='gre':
        return 7
    if description =='T2star':
        return 8
    if description =='dwi':
        return 9
    if description =='adc':
        return 10
    if description =='bold':
        return 11
    if description =='angio': #flowsensitive
        return 12
    if description =='pwi' or description=='cbf':
        return 13
    if 'asl' in description:
        return 14
    if description == 'hippo':
        return 15
    if description == 'dti':
        return 16
    return -1

In [33]:
CreateLabelsFrame['label'] = CreateLabelsFrame.apply(assignLabel,axis=1)

In [34]:
CreateLabelsFrame[CreateLabelsFrame['label']==-1]

,ID,label,extra,description,weighting,origDesc


In [35]:
CreateLabelsFrame = CreateLabelsFrame[CreateLabelsFrame['label']!=-1]

In [36]:
CreateLabelsFrame[['ID','label','extra']].to_csv(outname,index=False, sep='\t', header=False)

In [37]:
CreateLabelsFrame['label'].value_counts()

label
17    130925
8      90125
0      52200
15     45550
3      44925
13     21850
14      8200
16      1850
6        350
5         50
Name: count, dtype: int64